<a href="https://colab.research.google.com/github/araliadata/AraliaOpenRAG/blob/main/Interactive_Notebook_Understanding_AI_Powered_Tools_Development_and_Aralia_OpenRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*This documentation and associated AI-powered tools tutorial is released under the [MIT License](https://opensource.org/license/mit), which permits any type of modification and commercialization.*

# Interactive Notebook: Understanding AI-Powered Tools Development and Aralia OpenRAG

This interactive notebook guides you through the process of designing AI-powered tools tapping into the Aralia Open Data Ecosystem using the Aralia OpenRAG SDK. Each section provides sample code and explanations to facilitate your exploration and development of AI-powered tools.


Feel free to edit the code within this notebook, however, any changes will not be saved. If you'd like to save the changes, please first create a new copy.

Helpful comments are added to each code block (preceded with #). If you are required to input information, a comment (preceded with ##) will also inform you to do so. If this is your first time using this demo, we recommend following the flow of this demo, executing each code block one by one and fill in variables as needed.

If you would like to try out the results directly, please execute the cells under the **Setup** section, ensure there are no errors, and try it out in  **Section 2: AI-Powered Tools Workflow Execution** section.

### Aralia OpenRAG Developer's Guide

For setup instructions and other usage tips for the Aralia OpenRAG SDK, go over to the [Aralia OpenRAG Developer's Guide](https://deciduous-centipede-9d7.notion.site/AI-269ddf94fd148147b05dc28100ad84a8) for an overview.

## **Section 1: AI Workflow Tutorial: Sequential Tools**

If you need to implement more complex process control in AI Workflow applications, you can achieve this through the Sequential Tools model. This tutorial will demonstrate an example that starts with a user's question, retrieves real-time data through Aralia Data Planet, and generates a truly "evidence-based" answer for the user. This example uses a series of steps to help you understand how to leverage multiple Nodes in LangGraph, each with its own responsibility, and combine them into an intelligent and controllable multi-step process.

### Overall Process Description


1. Search Aralia Data Planet Datasets: Search for matching datasets in Data Planet using the user's question.
2. Generate Aralia Data Exploration Query: Use a language model to formulate a query statement based on the user's question and the identified datasets.
3. Refine Data Exploration Query: Improve precision and accuracy of the query by adding filter parameters  based on the user's question.
4. Execute Query: Connect to Data Planet using Aralia Tools, execute the generated query, and retrieve the results.
5. Respond to the User: Analyze the query results using a language model or custom logic, and provide a final response to the user


### **1.1 Install Aralia OpenRAG SDK and AI Development Tools**
Before you start using the Aralia OpenRAG SDK for your project, you will need to install and configure the necessary packages first. Follow these easy steps to quickly complete the installation and basic setup.

**Install the Aralia OpenRAG SDK**

Grab the Aralia OpenRAG SDK from GitHub.


In [ ]:
# Run this cell to install the Aralia OpenRAG SDK.

! pip install git+https://github.com/araliadata/AraliaOpenRAG.git

**Install required Python packages**

These are the additional packages needed for AI-powered tools.

In [ ]:
# Run this cell to install additional necessary packages.

! pip install langgraph dotenv langchain_openai langchain_anthropic langchain_google_genai

**Import Aralia OpenRAG and other necessary modules**

After the installation is complete, you can then import the necessary modules and they are now ready to be used:

In [ ]:
# Run this cell to import the necessary modules.

from nodes import SearchNode, PlanningNode, FilterDecisionNode, ExecutionNode, InterpretationNode
from core.graph import AraliaAssistantGraph
from core.state import GraphState, SearchResult
from core.config import AraliaConfig
from tools.aralia import AraliaClient

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

from typing import TypedDict, Dict, List, Any, Optional, Annotated
from operator import add
import pandas as pd
import os

### **1.2 Getting your favorite LLM API key (OpenAI, Gemini, Anthropic)**

*For the purpose of this demo, we are using Google's LLM, Gemini. Please set up using the Gemini API key. For advanced users, feel free to experiment with different LLMs.*


1. Follow the [instructions](https://colab.research.google.com/drive/1LKJVjt62qTrlw-cIIk4PNKF_ZVYJ1FTx#scrollTo=HF1F85Lx_v1U) and scroll down to the **Getting your favorite LLM API keys (OpenAI, Gemini, Anthropic)** section to get the API key to access your favorite LLM.

2. Open the "Secrets" tab on the left sidebar with thisimage.png icon.

3. Click "Add new secret" and copy and paste the following name(s) under **Name** to add to your list of secrets:

    a. `GEMINI_API_KEY`

    b. `OPENAI_API_KEY`

    c. `ANTHROPIC_API_KEY`

4. Enter the corresponding API key under **Value** in the Secrets tab.

### **1.3 Getting an Aralia user account**

You will need a valid Aralia account in order to use Aralia OpenRAG. Ensure that you have logged in to Aralia's platform before with this set of client ID and client secret.

1. Follow the [instructions](https://colab.research.google.com/drive/1LKJVjt62qTrlw-cIIk4PNKF_ZVYJ1FTx#scrollTo=Sr_4es38FPa1) to get your free account.

2. Open the "Secrets" tab on the left sidebar with thisimage.png icon.

3. Click "Add new secret" and copy and paste the following name(s) under **Name** to add to your list of secrets:

    `ARALIA_CLIENT_ID`

    `ARALIA_CLIENT_SECRET`

4. Enter the corresponding credentials under **Value** in the Secrets tab (`ARALIA_CLIENT_ID` is your Aralia AI client ID; `ARALIA_CLIENT_SECRET` is your Aralia AI client secret).

### **1.4 Setting credentials for your AI-powered tools**

Carefully read the [instructions](https://colab.research.google.com/drive/1LKJVjt62qTrlw-cIIk4PNKF_ZVYJ1FTx#scrollTo=7wOdR7dZAD1E) to set up the credentials for Aralia OpenRAG.

Then execute the following code snippet to finalize your environment.

In [ ]:
# Run this cell to set API keys/credentials.

if "COLAB_GPU" in os.environ:
    from google.colab import userdata

    def get_secret(key):
        try:
            secret = userdata.get(key)
            print(f"Retrieved secret '{key}' from Colab userdata!")
            return secret
        except userdata.SecretNotFoundError as e:
            print(
                f"The secret '{key}' was not found. "
                "If in Google Colab, add it via the key icon in the left sidebar."
            )
            return ""

    os.environ["GEMINI_API_KEY"] = get_secret('GEMINI_API_KEY')
    os.environ["OPENAI_API_KEY"] = get_secret('OPENAI_API_KEY')
    os.environ["ANTHROPIC_API_KEY"] = get_secret('ANTHROPIC_API_KEY')

    os.environ["ARALIA_CLIENT_ID"] = get_secret('ARALIA_CLIENT_ID')
    os.environ["ARALIA_CLIENT_SECRET"] = get_secret('ARALIA_CLIENT_SECRET')

else:
    from dotenv import load_dotenv
    load_dotenv()

### **1.5 Choose an Aralia API endpoint**

In [ ]:
# Run this cell to set the API endpoint URL.

sso_url = "https://marketing-sso.araliadata.io"
planet_url = "https://global-un.araliadata.io"

### **1.6 Choose a LLM**

In [ ]:
# Run this cell to set your LLM API key and model
# You can configure different providers:

# For Gemini (Google):
llm_api_key = os.getenv("GEMINI_API_KEY")
llm_model = "gemini-2.0-flash"

# For OpenAI:
## llm_api_key = os.getenv("OPENAI_API_KEY")
## llm_model = "gpt-4o"

# For Anthropic:
## llm_api_key = os.getenv("ANTHROPIC_API_KEY")
## llm_model = "claude-3-5-sonnet-20241022"

## **Section 2: AI-Powered Tools Workflow Execution**


The tool workflow is defined using the `AraliaAssistantGraph` class, which connects the nodes in a sequence.

The nodes are executed in the following sequence:
1. `aralia_search`: Searches for relevant datasets.
2. `analytics_planning`: Plans the analytics process.
3. `filter_decision`: Determines filtering criteria.
4. `analytics_execution`: Executes the analytics process.
5. `interpretation`: Generates the final response.


**Tool workflow overview**

5-tools.svg

### **Try it yourself**

**Run your node workflow**

In [ ]:
# Run this cell to see the outputs of the complete tool workflow.

## Enter your question within the quotation marks.
question = "Is there a relationship between the average GDP growth at purchasers' prices from 2021 to 2024 and gini coefficient of each state in Malaysia in 2024?"

# Create configuration
config = AraliaConfig(
    aralia_sso_url=sso_url,
    aralia_client_id=os.getenv("ARALIA_CLIENT_ID"),
    aralia_client_secret=os.getenv("ARALIA_CLIENT_SECRET"),
    aralia_stellar_url=planet_url,
    verbose=True,
    log_level="WARNING"
)

assistant_graph = AraliaAssistantGraph(config)

# Execute the workflow
response = assistant_graph.invoke({
    "question": question,
    "ai": llm_api_key,  # API key for LLM
})

## **Section 3: AI-powered tools Deep Dive**

Welcome! This tutorial section will walk you through how AI-powered tools work, step by step. We'll cover the important bits, like:

- What is "State"? We'll look at how nodes understand their situation.
- What are Nodes: Think of these as the action points or decision steps for a node workflow.
- Writing good prompts: Learn how to ask the LLM the right things to get the results you want.
- Node steps and why they matter: We'll break down what each node does and its job in the bigger picture.

### **3.1 State Definition**


The `GraphState` class is used to manage the state of the system as it progresses through different nodes. It includes the following fields:



**Input parameters:**
- **`question`**: The user's query.
- **`ai`**: The LLM instance for processing.
- **`config`**: Configuration dictionary.


**Aralia-specific inputs:**
- **`aralia_sso_url`**: Aralia SSO URL.
- **`aralia_stellar_url`**: Aralia Stellar URL.
- **`aralia_client_id`**: Aralia client ID.
- **`aralia_client_secret`**: Aralia client secret.


**Processing state:**
- **`search_results`**: List of search results from dataset discovery.
- **`selected_datasets`**: Datasets selected for analysis.
- **`query_plan`**: Generated analytics plan.
- **`filter_options`**: Available filter options for datasets.


**Output:**
- **`response`**: Intermediate response from nodes.
- **`final_response`**: Final formatted response.


**System state:**
- **`errors`**: List of errors encountered during execution.
- **`execution_metadata`**: Metadata about the execution process.
- **`at`**: AraliaClient instance for interacting with the Aralia platform.
- **`verbose`**: Flag to enable or disable verbose output.


The state is passed between nodes to maintain context and data throughout the workflow. Each node updates the state with its results, ensuring that subsequent nodes have access to the necessary information.

In [ ]:
class GraphState(TypedDict):
    """LangGraph state definition following official best practices.

    This state object maintains all information needed throughout the
    graph execution, including inputs, intermediate results, and outputs.
    """

    # Input parameters
    question: str  # User's question
    ai: Any  # LLM instance (will be set by graph)
    config: Dict[str, Any]  # Configuration dictionary

    # Aralia-specific inputs
    aralia_sso_url: Optional[str]
    aralia_stellar_url: Optional[str]
    aralia_client_id: Optional[str]
    aralia_client_secret: Optional[str]

    # Processing state
    search_results: Annotated[List[SearchResult], add]
    selected_datasets: Optional[List[Dict[str, Any]]]
    query_plan: Optional[Dict[str, Any]]
    filter_options: Optional[List[Dict[str, Any]]]

    # Output
    response: Optional[Any]  # Intermediate response
    final_response: Optional[str]  # Final formatted response

    # System state
    errors: Annotated[List[str], add]
    execution_metadata: Dict[str, Any]

    # Required for current implementation
    at: Optional[Any]  # AraliaClient instance
    verbose: Optional[bool]

### **3.2 Node Definitions**


Nodes are modular functions that perform specific tasks in the workflow. Each node takes the `state` as input, processes it, and returns an updated state.


#### **3.2.1 `aralia_search`**

- **Purpose**: Searches for datasets on the Aralia platform that are relevant to the user's query.
- **Key Steps**:
  1. Uses the `search_datasets` method from `AraliaClient` to retrieve datasets.
  2. Filters datasets using a prompt template (`DATASET_EXTRACT`) and an AI model.
  3. Handles retries to ensure robust dataset extraction.
- **Output**: Updates the GraphState with `response` containing the filtered datasets.


##### **Prompt**

The default `aralia_search` prompt:

```
You are an expert data analyst tasked with filtering datasets based on relevance to a user's question.

**Task**: For the following question, identify and retain only the most directly relevant datasets.
Remove any datasets that are indirect, redundant, or tangentially related.

**Question**: {question}

**Available Datasets**: {datasets}

**Instructions**:
1. Analyze the core intent and key entities in the question
2. Evaluate each dataset's relevance to answering the question
3. Prioritize datasets that directly contain the required information
4. Remove datasets that are only tangentially related
5. Aim for quality over quantity - better to have fewer highly relevant datasets

Return the dataset keys and names for the most relevant datasets only.
```

##### **Try it yourself**


**Run your node**

In [ ]:
# Run this cell to see the outputs of the aralia_search node.

## Enter your question within the quotation marks.
question  = "What is the average GDP growth rate of each state in Malaysia in 2019?"

# Create Aralia client
aralia_client = AraliaClient(
    client_id=os.getenv("ARALIA_CLIENT_ID"),
    client_secret=os.getenv("ARALIA_CLIENT_SECRET"),
    sso_url=sso_url,
    stellar_url=planet_url
)

# Define state
state = GraphState(
    question=question,
    at=aralia_client,
    # ai=ChatAnthropic(api_key=os.environ["ANTHROPIC_API_KEY"], model="claude-3-5-sonnet-20240620", temperature=0),
    # ai=ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"], model="gpt-4o", temperature=0),
    ai=ChatGoogleGenerativeAI(api_key=llm_api_key, model=llm_model, temperature=0), # Please use Gemini for a smooth experience. Advanced users may try with a different LLM.
    verbose=True,
)

# Run the node
response1 = SearchNode().execute(state)
# response1

#### **3.2.2 `analytics_planning`**


- **Purpose**: Plans the analytics process by selecting relevant columns and chart specifications for the datasets.
- **Key Steps**:
  1. Retrieves column metadata for the datasets using `get_dataset_metadata`.
  2. Uses a prompt template (`CHART_PLOTTING`) to generate chart specifications.
  3. Filters and formats the datasets based on the AI response.
  4. Handles retries for robust processing.
- **Output**: Updates the GraphState with `response` containing the filtered datasets and chart specifications.

##### **Prompt**

The default `analytics_planning` prompt:

```
# ROLE AND OBJECTIVE
You are a senior data analyst expert, skilled in data exploration, correlation analysis, and effective data visualization design.

**Objective**: Based on the user's question, analyze each provided dataset and propose **only one specific chart proposal per relevant dataset** that most effectively answers the question.

# INPUT INFORMATION
- **Question:** {question}
- **Datasets:** {datasets} (includes dataset descriptions, column names, column types, and metadata)
- **Administrative Levels:** {admin_level}

# ANALYSIS FRAMEWORK
Execute the following phases systematically, documenting your thought process for each step:

## Phase 1: Problem Analysis
**Deep Question Understanding:**
- Analyze the user's question intent and break it down into components
- Identify key entities, metrics, and analytical requirements

**Dimension Classification (Critical):**
Distinguish between two types of dimensions:

1. **主軸維度 (Primary Axis Dimension):**
   - Forms the chart's main axis (typically x-axis)
   - Examples: time series, continuous numerical ranges
   - Usually only one per chart

2. **分類比較維度 (Grouping/Comparison Dimension):**
   - Categorical fields for grouping/comparing metrics
   - Examples: store names, product types, geographic regions
   - Can have multiple dimensions

**Documentation:** Record your analysis explicitly listing:
- Identified metrics
- Primary axis dimensions  
- Grouping/comparison dimensions

## Phase 2: Dataset Curation
- Retain only the most relevant datasets for answering the question
- Remove datasets that are indirect, redundant, or tangentially related
- Prioritize quality over quantity

## Phase 3: Column Selection
- For each selected dataset, identify the **minimum necessary columns** to answer the question
- Focus on essential fields that directly contribute to the analysis

## Phase 4: Chart Specification (Per Dataset)
**Component Identification:**
Based on Phase 1 analysis, identify:
- **Metrics (指標):** Quantitative fields for measurement (y-axis candidates)
- **Primary Axis Dimension (主軸維度):** Main dimension for x-axis
- **Grouping/Comparison Dimension (分類比較維度):** Dimensions for data breakdown

**Axis Assignment Rules:**
- **X-axis:** Fields of type `date/datetime/space/nominal/ordinal/point/line/polygon`
  (when serving as Primary Axis or Grouping/Comparison Dimension)
- **Y-axis:** Fields of type `integer/float` (when serving as Metrics)

## Phase 5: Filter Specification (Per Dataset)
**Filter Categories:**
- **Temporal Scope:** Date/time ranges (when applicable)
- **Spatial Boundaries:** Geographic constraints (when applicable)  
- **Category Filters:** Specific categorical values (when applicable)

**Dual-Purpose Fields:** Include fields that serve both as chart dimensions and filter criteria

## Phase 6: Format and Calculation Rules (Per Dataset)

**Date/DateTime Fields:**
- `format` options: `["year", "quarter", "month", "week", "date", "day", "weekday", "year_month", "year_quarter", "year_week", "month_day", "day_hour", "hour", "minute", "second", "hour_minute", "time"]`
- `operator`: `"in"`

**Spatial Fields (space/point/line/polygon):**
- `format`: Use most general `admin_level_x` (lowest number) based on user question context

**Nominal/Integer/Float Fields:**
- `format`: `""` (empty string)

**Calculation Methods:**
- **Integer/Float:** `["count", "sum", "avg", "min", "max", "distinct_count"]`
- **Nominal:** `["count", "distinct_count"]`

## Phase 7: Output Generation
**Array Construction Rules:**
- **`x` array:** Include Primary Axis Dimension + any Grouping/Comparison Dimensions
- **`y` array:** Include identified Metrics with appropriate calculations
- **`filter` array:** Apply Phase 5 specifications

**Important:** Grouping/Comparison Dimensions often appear in both `x` array (for grouping) and `filter` array (for category selection)

# OUTPUT FORMAT
Return results in the following JSON structure:

```json
{{
    "charts": [
        {{
            "id": "dataset_id",
            "name": "dataset_name",
            "x": [
                {{
                    "columnID": "column_id",
                    "name": "field_displayName",
                    "type": "field_type",
                    "format": "format_specification"
                }}
            ],
            "y": [
                {{
                    "columnID": "column_id",
                    "name": "field_displayName",
                    "type": "field_type",
                    "calculation": "aggregate_function"
                }}
            ],
            "filter": [
                {{
                    "columnID": "column_id",
                    "name": "field_name",
                    "calculation": "aggregate_function",
                    "type": "field_type",
                    "format": "format_specification",
                    "operator": "filter_operator",
                    "value": ["filter_value"]
                }}
            ]
        }}
    ]
}}
```

##### **Try it yourself**


In [ ]:
## Run this cell to see the outputs of the analytics_planning.

response2 = PlanningNode().execute(state | response1)
response2['response']

#### **3.2.3 `filter_decision`**


- **Purpose**: Determines the filtering criteria for the datasets based on the user's query.
- **Key Steps**:
  1. Uses the `get_filter_options` method from `AraliaClient` to retrieve filter options.
  2. Generates a filtering prompt using `QUERY_GENERATION`.
  3. Processes the AI response to finalize the filtering criteria.
- **Output**: Updates the GraphState with `response` containing the filtered datasets and their filtering criteria.


##### **Prompt**

The default `filter_decision` prompt:

```
You are a senior data analyst specializing in statistical data analysis and query optimization.

## Task
Generate optimized query configurations based on the user's question and available dataset structures.

## Input
- **User Question**: {question}
- **Dataset Configurations**: {response}

## Instructions

### Core Rules
1. **Preserve Structure**: Maintain exact same top-level keys and array structures
2. **Modify Only**: `operator` and `value` fields in filter objects
3. **No Additions**: Do not add new filter objects or modify other fields
4. **No Removals**: Do not remove existing filter objects

### Operator Selection
- **Date/DateTime/Nominal/Spatial**: Use `"in"` operator
- **Integer/Float**: Use `"range"`, `"lt"`, `"gt"`, `"lte"`, or `"gte"`

### Value Specification
- Analyze the user question carefully to determine appropriate filter values
- For geographic locations, consider administrative boundaries and actual locations
- For nominal categories, use exact matches from available options
- For date ranges, specify appropriate time periods

### Special Considerations
- Some institutions may be named after one location but physically located elsewhere
- Example: "Taipei Motor Vehicles Office" is actually in New Taipei City
- Always verify geographic relationships when setting spatial filters

Return the modified configuration with updated `operator` and `value` fields only.
```

##### **Try it yourself**

In [ ]:
## Run this cell to see the outputs of the filter_decision.

response3 = FilterDecisionNode().execute(state | response2)
response3['response']

#### **3.2.4 `analytics_execution`**


- **Purpose**: Executes the analytics process by exploring the datasets and generating visualizations.
- **Key Steps**:
  1. Uses the `execute_exploration` method from `AraliaClient` to retrieve and process data.
  2. Generates CSV files of the datasets.
- **Output**: Updates the GraphState with `search_results` containing the processed datasets.


##### **Try it yourself**

In [ ]:
## Run this cell to see the outputs of the analytics_execution.

response4 = ExecutionNode().execute(state | response3)
response4

#### **3.2.5 `interpretation`**


- **Purpose**: Interprets the results of the analytics process and generates a final response for the user.
- **Key Steps**:
  1. Constructs a message for the AI model with the user's query and the processed data.
  2. Invokes the AI model to generate a concise, 300-word summary.
- **Output**: Updates the GraphState with `final_response` containing the AI-generated summary.


##### **Prompt**

The default `interpretation` prompt:

```
You are an expert data analyst providing insights based on retrieved data.

## Task
Analyze the provided data and answer the user's question with clear, actionable insights.

## Input
- **Question**: {question}
- **Data Results**: {search_results}

## Requirements

### Language Matching
**CRITICAL**: Your response language MUST exactly match the language used in the question.

### Content Requirements
1. **Detailed Analysis**: Provide thorough analysis of the charts and data patterns
2. **Direct Answer**: Give a clear, direct answer to the specific question asked
3. **Concise Conclusion**: Summarize key findings in under 300 words
4. **Evidence-Based**: Base all conclusions on the actual data provided in "json_data"

### Structure
1. **Data Overview**: Brief summary of the datasets analyzed
2. **Key Findings**: Main insights from the data
3. **Direct Answer**: Specific answer to the user's question
4. **Conclusion**: Summary of implications and recommendations

Focus on actionable insights and ensure all claims are supported by the provided data.
```

**Advanced option: Set your own prompt for the interpretation node.**

In [ ]:
## Uncomment the following line of code and run this cell to overide the default prompt with your own

# state["interpretation_prompt"] = "replace this text with custom prompt"

In [ ]:
## Uncomment the following line of code to revert to the default prompt

# state["interpretation_prompt"] = None

##### **Try it yourself**

In [ ]:
## Run this cell to see the outputs of the interpretation.

response5 = InterpretationNode().execute(state | response4)
response5

### **3.3 Error Handling**


Each node includes error handling mechanisms to ensure robustness:
  - **Retries**: Nodes like `aralia_search`, `analytics_planning`, and `filter_decision` retry operations up to 5 times.
  - **Exceptions**: Nodes raise `RuntimeError` if critical operations fail (e.g., dataset retrieval, AI response generation).
  - **Verbose output**: Step by step processes are printed to the console if `state['verbose']` is set to `True`.

### **3.4 Integration with Aralia APIs**

The `AraliaClient` class provides methods for interacting with Aralia's APIs, such as searching for datasets, retrieving column metadata, and exploring data.

In [ ]:
# Initialize Aralia Client
aralia_client = AraliaClient(
    client_id=os.getenv("ARALIA_CLIENT_ID"),
    client_secret=os.getenv("ARALIA_CLIENT_SECRET"),
    sso_url=sso_url,
    stellar_url=planet_url
)

# Search for datasets
datasets = aralia_client.search_datasets("What is the average GDP growth rate of each state in Malaysia in 2019?")
pd.DataFrame(datasets)

### **3.5 Conclusion**


This deepdive demonstrates the modular design of the AI-powered tools and its integration with Aralia's APIs. By running the provided code snippets, you can explore the functionality of each component interactively.